In [3]:
%run library_TS.py

Python version: 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]
NumPy version: 1.23.5
pandas version: 1.5.3
matplotlib: 3.7.1
TSfresh: 0.20.0
Sklearn version: 1.1.3
Xgboost version: 1.5.0
Lightgbm version: 2.2.3


c:\Users\Windows\miniconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# LOAD THE TIMESERIES AND GROUP THEN IN A DICTIONNARY

The first step to make feature extraction is to load the TS files (containing in different folder and subfolder), and group them by variable type (RR, cardiac rate...) and by index (in the name of the files). To do that we make a dictionnary, including in the key the index. 

In [4]:
# Defintion of the master folder path containing all the TS files
parent_folder = "C:/Users/Windows/Documents/Essai/TS_truncate_24H"

# All patients list
folder_path = "C:/Users/Windows/Documents/Essai/TS_truncate_24H/Hemodynamique/Hourly_Diuresis/"

lst_pat = [os.path.splitext(filename)[0] for filename in os.listdir(folder_path)]
lst_pat.sort()
txt_files = []


for root, dirs, files in os.walk(parent_folder):
    # Filtering the txt files and add their name in a list
    txt_files += [os.path.join(root, file) for file in files if file.endswith('.txt')]
# Dictionnary initialisation for containing all the df for every patient
dico_df  = {}

# For every txt find :
for pat in lst_pat:
   df_temp = pd.DataFrame()
   for file in [i for i in txt_files if pat in i]:
        # Rename
        variable = file[file.rfind("\\", 0, file.rfind("\\") - 1) + 1:file.rfind("\\")]
        try :
            temp = pd.read_csv(file, delimiter='\t', header=None, index_col=0).iloc[:,0:1].rename(columns = {1: variable})
            temp.index = pd.to_datetime(temp.index)
            if temp[temp.columns[0]].dtypes == 'object': # Transformation of the object variable (with numb >999) in float
                temp.iloc[:, 0] = temp.iloc[:, 0].apply(lambda x: re.sub(r'\s+', '', x))
            temp[temp.columns[0]] = temp[temp.columns[0]].astype('float')
            #temp.iloc[:, 0] = temp.iloc[:, 0].astype('float')
            #temp = temp.astype({"": int, "Proverty_rate": float, "Median_Age": int }) 0].str.replace(' ', '')
        except EmptyDataError:
            temp = pd.DataFrame()
            #temp = pd.DataFrame({variable: [0]})
        #lst_df = lst_df.append(temp)
        df_temp = pd.concat([df_temp,temp], ignore_index=False, axis=1)
   if not df_temp.empty:
       df_temp = df_temp.groupby(df_temp.index.floor('H'), group_keys=False).apply(lambda s: s.ffill().bfill())
       df_temp.fillna(method='ffill', inplace=True)
       df_temp.fillna(df_temp.median(), inplace=True)
   dico_df[pat] = df_temp

print(len(dico_df))
for key, df in dico_df.items():
    print(f"DataFrame {key} : {df.shape[0]} lignes, {df.shape[1]} colonnes")

213
DataFrame 0001 : 1440 lignes, 16 colonnes
DataFrame 0003 : 1440 lignes, 16 colonnes
DataFrame 0005 : 1440 lignes, 16 colonnes
DataFrame 0006 : 1440 lignes, 16 colonnes
DataFrame 0008 : 1440 lignes, 16 colonnes
DataFrame 0009 : 1440 lignes, 16 colonnes
DataFrame 0011 : 1440 lignes, 16 colonnes
DataFrame 0013 : 1440 lignes, 16 colonnes
DataFrame 0014 : 1440 lignes, 16 colonnes
DataFrame 0015 : 1440 lignes, 16 colonnes
DataFrame 0016 : 1440 lignes, 16 colonnes
DataFrame 0017 : 1440 lignes, 16 colonnes
DataFrame 0018 : 548 lignes, 13 colonnes
DataFrame 0019 : 1388 lignes, 15 colonnes
DataFrame 0020 : 1440 lignes, 16 colonnes
DataFrame 0021 : 1440 lignes, 16 colonnes
DataFrame 0022 : 1440 lignes, 13 colonnes
DataFrame 0023 : 1440 lignes, 16 colonnes
DataFrame 0025 : 286 lignes, 15 colonnes
DataFrame 0026 : 1440 lignes, 12 colonnes
DataFrame 0027 : 1440 lignes, 16 colonnes
DataFrame 0028 : 1440 lignes, 16 colonnes
DataFrame 0029 : 1440 lignes, 14 colonnes
DataFrame 0030 : 1440 lignes, 16

# PREPARE THE DICTIONNARY TO FEATURE EXTRACTION

In this step, the library need to import the label csv, and to convert the dictionnary into a dataframe

In [5]:
# Importation of the target csv 
target_df = pd.read_excel("//wsl.localhost/Ubuntu/home/romain/GITHUB/Extub-IA/BDD/Target.xls", dtype=str, index_col=None, usecols=["ID", "Weaning_success"])

# Definition of the ID
sequence_ids = target_df['ID']
# Definition of the label
labels = target_df['Weaning_success']

X = pd.DataFrame()
y = labels

# Fill the X dataset with the dictionnary created previously
for i, sequence in enumerate(sequence_ids):
    df = dico_df[sequence].copy()
    df.insert(0, 'sequence', i)
    df['step'] = np.arange(df.shape[0]) # creates a range of integers starting from 0 to the number of the measurements.
    X = pd.concat([X, df])
print(f"The length of X dataset is {len(X)}")
print(f"The length of target is {len(y)}")


The length of X dataset is 295772
The length of target is 213


In [6]:
# Display the missing value
print(f'Missing value before imputation:\n{X.isna().sum()}')
print(f'-----------------------------------')

# Replace the missing value by the mean
X = X.fillna(X.mean())

# Display the missing value
print(f'Missing value after imputation::\n{X.isna().sum()}')

Missing value before imputation:
sequence                   0
CR                      1440
Cumulative_Diuresis     2880
DBP                    11829
Hourly_Diuresis            0
MAP                    11829
SBP                    13269
Glycemia                 286
CPIS                    2880
Temperature             2880
RASS                   65474
FiO2                       0
Minute_Volume              0
PEEP                       0
RR                      7113
SpO2                       0
Tidal_Volume            5760
step                       0
dtype: int64
-----------------------------------
Missing value after imputation::
sequence               0
CR                     0
Cumulative_Diuresis    0
DBP                    0
Hourly_Diuresis        0
MAP                    0
SBP                    0
Glycemia               0
CPIS                   0
Temperature            0
RASS                   0
FiO2                   0
Minute_Volume          0
PEEP                   0
RR           

# EXTRACTION FEATURE 

After all the data are prepared, we can extract the feature. For this we used TSFresh. Many calculations are made and take time. 

In [7]:
# Extraction all the features for X dataset
# The parameters default_fc_parameters=xxx permit to select the feature we want
# https://stackoverflow.com/questions/50426458/retrieve-specific-features-by-using-tsfresh-in-python

time_start = time.perf_counter()
extracted_features = extract_features(X, column_id='sequence', column_sort='step', n_jobs= 10)
#n_jobs definites the parallelisation

time_elapsed = (time.perf_counter()-time_start)
print("extraction done in %.2f seconds" %(time_elapsed))


Feature Extraction: 100%|██████████| 50/50 [3:26:33<00:00, 247.87s/it]  


extraction done in 12395.06 seconds


In [8]:
# We can also define the feature we want, like mean, max and min, variance : 
fc_parameters = {
    "mean": None,
    "maximum":None, 
    "minimum":None, 
    "variance":None
}

time_start = time.perf_counter()

extracted_features2 = extract_features(X, column_id='sequence', column_sort='step', n_jobs= 10, default_fc_parameters =fc_parameters)

time_elapsed = (time.perf_counter()-time_start)
print("extraction done in %.2f seconds" %(time_elapsed))

extracted_features2.to_csv('Minimal Feature extraction.csv', index=True)

Feature Extraction: 100%|██████████| 50/50 [00:09<00:00,  5.09it/s]


extraction done in 10.06 seconds


Now we can save the full extract dataset

In [9]:
extracted_features.to_csv('All features extracted.csv')

After extraction we can made another imputation, with the function impute() of TSfresh

In [10]:
# Imputation with TSFresh
impute(extracted_features)

# Reset the index
y = y.reset_index()

c:\Users\Windows\miniconda3\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['RR__friedrich_coefficients__coeff_0__m_3__r_30'
 'RR__friedrich_coefficients__coeff_1__m_3__r_30'
 'RR__friedrich_coefficients__coeff_2__m_3__r_30'
 'RR__friedrich_coefficients__coeff_3__m_3__r_30'
 'RR__max_langevin_fixed_point__m_3__r_30'
 'RR__query_similarity_count__query_None__threshold_0.0'
 'SpO2__friedrich_coefficients__coeff_0__m_3__r_30'
 'SpO2__friedrich_coefficients__coeff_1__m_3__r_30'
 'SpO2__friedrich_coefficients__coeff_2__m_3__r_30'
 'SpO2__friedrich_coefficients__coeff_3__m_3__r_30'
 'SpO2__max_langevin_fixed_point__m_3__r_30'
 'SpO2__query_similarity_count__query_None__threshold_0.0'
 'Tidal_Volume__query_similarity_count__query_None__threshold_0.0'
 'CR__query_similarity_count__query_None__threshold_0.0'
 'Cumulative_Diuresis__friedrich_coefficients__coeff_0__m_3__r_30'
 'Cumulative_Diuresis__friedrich_coefficients__coeff_1__m_3__r_30'
 'Cumulati

# DIMENSION REDUCTION

Now we have XX new column extract. It's too big to make good prediction and classification. So we made dimension reduction, by the FRESH method. Consisting in take only the relevant features determined with a univariate statistic test. After this calculation, all the p-value are corrected with the Benjamini-Hochberg method and we take only the XX first one, or the p-value <0.05

In [11]:
print(f"Number of features extracted :",len(extracted_features.columns))

Number of features extracted : 12608


In [12]:
y_label = y['Weaning_success'].squeeze()

# Calculate the relevance table https://tsfresh.readthedocs.io/en/latest/text/feature_filtering.html
time_start = time.perf_counter()
relevance_table = calculate_relevance_table(extracted_features, y_label, ml_task='classification')
#relevance_table = relevance_table[relevance_table.relevant] only if we have enought data
relevance_table.sort_values("p_value", inplace=True) # Sort by ascending P-value

#list_feature = relevance_table["feature"][relevance_table.p_value<0.05] # Take only the feature with p-value<0.05
list_feature = relevance_table["feature"][:150] # or take the 150th first columns

filter_features = extracted_features.loc[:,list_feature]

time_elapsed = (time.perf_counter()-time_start)
print("extraction done in %.2f seconds" %(time_elapsed))

print(f'nombre de colonne du df filtré:', len(filter_features.columns))

extraction done in 10.16 seconds
nombre de colonne du df filtré: 150


In [13]:
# Save the filter features extracted in csv
filter_features.to_csv('Filtered features extracted (150 features).csv')